# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [22]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [23]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
# for root, dirs, files in os.walk(filepath):
#     print(root)
# join the file path and roots with the subdirectories using glob
file_path_list = glob.glob(os.path.join(filepath,'*.csv'))
#     print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [24]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [25]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [26]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [27]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [28]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [87]:
## Creating a table that houses the session ID, the item in session, the artist name, the song title and length.
## This table was created this way so as to quickly get the artist name, song title and song length knowing
## the session ID and item in session.The composite key was selected as sessionId and itemInSession to allow the user 
## to query more specifically

query = "CREATE TABLE IF NOT EXISTS music_library_item"
query = query + "(sessionId int, \
                  itemInSession int, \
                  artist_name text, \
                  song_title text, \
                  song_length float, \
                  PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
## This table follows the same logic as the above table. These columns were created in order to get the artist name, 
## song title, and the user's name while knowing the user id and session id. Then, we order on itemInSession. 

query = "CREATE TABLE IF NOT EXISTS music_library_users"
query = query + "(userId int, \
                  sessionId int, \
                  artist_name text, \
                  first_name text, \
                  last_name text, \
                  song_title text, \
                  itemInSession int, \
                  PRIMARY KEY ((userId, sessionId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
## The table here allows us to get the first and last name of a user based on what song they've listened to
## The primary composite key here is the song title, userID, which allows for querying on both the song title and 
## the user ID. 
    
query = "CREATE TABLE IF NOT EXISTS music_library_who"
query = query + "(song_title text, \
                  userId int, \
                  first_name text, \
                  last_name text, \
                  PRIMARY KEY (song_title, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [88]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_library_item (sessionId, \
                                                 itemInSession, \
                                                 artist_name, \
                                                 song_title, \
                                                 song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        ## Converting some values, as they would be more suitable to be in this format in the database
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        
        query = "INSERT INTO music_library_users (userId, \
                                                  sessionId, \
                                                  artist_name,\
                                                  first_name,\
                                                  last_name, \
                                                  song_title, \
                                                  itemInSession)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Converting some values, as they would be more suitable to be in this format in the database
        session.execute(query, (int(line[10]), int(line[8]), line[0], line[1], line[4], line[9], int(line[3])))
        
        query = "INSERT INTO music_library_who (userId, \
                                                first_name, \
                                                last_name, \
                                                song_title)"
        query = query + " VALUES (%s, %s, %s, %s)"
        ## Converting some values, as they would be more suitable to be in this format in the database
        session.execute(query, (int(line[10]), line[1], line[4], line[9]))

#### Do a SELECT to verify that the data have been inserted into each table

In [90]:
## Selecting based on session ID and item in session from the suitable table "music library item"

query = "SELECT artist_name, \
                song_title, \
                song_length \
         FROM music_library_item \
         WHERE sessionId = 338 \
         AND itemInSession = 4" 
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row[0], row[1], row[2])

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [83]:
## Selecting based on userID and session ID from the suitable table "music library users"

query = 'SELECT artist_name, \
                first_name, \
                last_name, \
                song_title \
         FROM music_library_users \
         WHERE userId = 10 \
         AND sessionId = 182'
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row[0], row[1], row[2], row[3])

                    

Down To The Bone Sylvie Cruz Keep On Keepin' On
Three Drives Sylvie Cruz Greece 2000
Sebastien Tellier Sylvie Cruz Kilometer
Lonnie Gordon Sylvie Cruz Catch You Baby (Steve Pitron & Max Sanna Radio Edit)


In [91]:
## Selecting the first and last name based on the song a user has listened to from the suitable table "music library who"

query = "SELECT first_name, \
                last_name \
         FROM music_library_who \
         WHERE song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row[0], row[1])
                    

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [86]:
## Dropping all the tables

query = 'DROP TABLE IF EXISTS music_library_item'
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = 'DROP TABLE IF EXISTS music_library_users'
try:
    session.execute(query)
except Exception as e:
    print(e)
    
    
query = 'DROP TABLE IF EXISTS music_library_who'
try:
    session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [33]:
session.shutdown()
cluster.shutdown()